In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cites-wildlife-trade-database/comptab_2018-01-29 16_00_comma_separated.csv


# Step 0: Load data 

In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import networkx as nx

# Load data
data_path = '/kaggle/input/cites-wildlife-trade-database/comptab_2018-01-29 16_00_comma_separated.csv'
df = pd.read_csv(data_path)

# Filter necessary columns
trade_data = df[['Importer', 'Exporter']]

# Create a graph
G = nx.Graph()
for _, row in trade_data.iterrows():
    G.add_edge(row['Importer'], row['Exporter'])

    
    
    
# ...

# 3D Network Graph
pos = nx.spring_layout(G, dim=3, seed=42)

# Extract node and edge positions from the layout
node_xyz = np.array([pos[v] for v in G.nodes()])  # Use G.nodes() instead of sorting
edge_xyz = np.array([(pos[u], pos[v]) for u, v in G.edges()])

# Create scatter trace for nodes
node_trace = go.Scatter3d(x=node_xyz[:,0], y=node_xyz[:,1], z=node_xyz[:,2],
                          mode='markers', marker=dict(size=5, color='blue'))

# ...

# Rest of the script remains the same

# # 3D Network Graph
# pos = nx.spring_layout(G, dim=3, seed=42)

# # Extract node and edge positions from the layout
# node_xyz = np.array([pos[v] for v in sorted(G)])
# edge_xyz = np.array([(pos[u], pos[v]) for u, v in G.edges()])

# # Create scatter trace for nodes
# node_trace = go.Scatter3d(x=node_xyz[:,0], y=node_xyz[:,1], z=node_xyz[:,2],
#                           mode='markers', marker=dict(size=5, color='blue'))

# Create line traces for edges
edge_traces = []
for i, j in edge_xyz:
    edge_traces.append(go.Scatter3d(x=[i[0], j[0]], y=[i[1], j[1]], z=[i[2], j[2]],
                                    mode='lines', line=dict(width=1, color='gray')))

# Create layout and figure
layout = go.Layout(
    title="3D Trade Network Visualization",
    scene=dict(
        xaxis=dict(title='Dimension X'),
        yaxis=dict(title='Dimension Y'),
        zaxis=dict(title='Dimension Z'),
    ),
    margin=dict(l=0, r=0, b=0, t=0)  # tight layout
)

fig = go.Figure(data=[node_trace] + edge_traces, layout=layout)
fig.show()


# Step 1: Import and Export Countries as Nodes, Transactions as Edges


In [3]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import networkx as nx

# Load data
data_path = '/kaggle/input/cites-wildlife-trade-database/comptab_2018-01-29 16_00_comma_separated.csv'
df = pd.read_csv(data_path)

# Filter necessary columns
trade_data = df[['Importer', 'Exporter']]

# Create a graph
G = nx.Graph()
for _, row in trade_data.iterrows():
    G.add_edge(row['Importer'], row['Exporter'])

# 3D Network Graph
pos = nx.spring_layout(G, dim=3, seed=42)

# Extract node and edge positions from the layout
node_xyz = np.array([pos[v] for v in G.nodes()])
edge_xyz = np.array([(pos[u], pos[v]) for u, v in G.edges()])

# Create scatter trace for nodes with text labels
node_trace = go.Scatter3d(
    x=node_xyz[:, 0], 
    y=node_xyz[:, 1], 
    z=node_xyz[:, 2],
    mode='markers+text',
    marker=dict(size=5, color='blue'),
    text=list(G.nodes()),  # Assign country names as labels
    hoverinfo='text'  # Show country names when hovering over the nodes
)

# Create line traces for edges
edge_traces = []
for i, j in edge_xyz:
    edge_traces.append(go.Scatter3d(
        x=[i[0], j[0]], 
        y=[i[1], j[1]], 
        z=[i[2], j[2]],
        mode='lines', 
        line=dict(width=1, color='gray'))
    )

# Create layout and figure
layout = go.Layout(
    title="3D Trade Network Visualization",
    scene=dict(
        xaxis=dict(title='Dimension X'),
        yaxis=dict(title='Dimension Y'),
        zaxis=dict(title='Dimension Z'),
    ),
    margin=dict(l=0, r=0, b=0, t=0)  # tight layout
)

fig = go.Figure(data=[node_trace] + edge_traces, layout=layout)
fig.show()


In [4]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import plotly.graph_objs as go
import networkx as nx
from sklearn.preprocessing import OneHotEncoder

# Load data
data_path = '/kaggle/input/cites-wildlife-trade-database/comptab_2018-01-29 16_00_comma_separated.csv'
df = pd.read_csv(data_path)

# Filter necessary columns and drop duplicates
trade_data = df[['Importer', 'Exporter']].drop_duplicates()

# Create a graph
G = nx.Graph()
for _, row in trade_data.iterrows():
    G.add_edge(row['Importer'], row['Exporter'])

# Prepare data for t-SNE: One-hot encode the country connections
encoder = OneHotEncoder()
encoded_data = encoder.fit_transform(trade_data).toarray()

# Apply t-SNE
tsne = TSNE(n_components=3, perplexity=30, n_iter=3000, random_state=42)
tsne_results = tsne.fit_transform(encoded_data)

# Extract the results
x = tsne_results[:, 0]
y = tsne_results[:, 1]
z = tsne_results[:, 2]

# Create scatter trace for nodes with text labels
node_trace = go.Scatter3d(
    x=x, y=y, z=z,
    mode='markers',
    marker=dict(size=5, color='blue'),
    text=encoder.categories_[0],  # Country names for each point
    hoverinfo='text'
)

# Create layout and figure
layout = go.Layout(
    title="3D t-SNE Visualization of Trade Data",
    scene=dict(
        xaxis=dict(title='t-SNE Dimension 1'),
        yaxis=dict(title='t-SNE Dimension 2'),
        zaxis=dict(title='t-SNE Dimension 3'),
    ),
    margin=dict(l=0, r=0, b=0, t=0)
)

fig = go.Figure(data=[node_trace], layout=layout)
fig.show()
